1. Parameter Accuracy

This part will check how accurate the LLM parse the parameters for the function call after reading the query.

2. delivery rate

In [3]:
import json
#delivery rate
def getDeliveryRate(model, numPlan):
    delivered = 0
    with open (f'Output/{model}/plans/toolUsePlans.jsonl','r') as f:
         plans = [json.loads(line.strip()) for line in f]

    for record in plans:
        if record['plan'] != "":
            delivered += 1
    return delivered / numPlan

In [8]:
deliverayRate = getDeliveryRate('llama318b',300)
deliverayRate

0.8466666666666667

In [ ]:
#get the error list, rerun, wrong parameter, we don't have max steps. 